In [1]:
 import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
# Load the data
file_path = Path('/Resources/lending_data.csv')
df = pd.read_csv(file_path)
df.head()

,loan_size,interest_rate,homeowner,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,own,52800,0.431818,5,1,22800,low_risk
1,8400.0,6.692,own,43600,0.311927,3,0,13600,low_risk
2,9000.0,6.963,rent,46100,0.349241,3,0,16100,low_risk
3,10700.0,7.664,own,52700,0.430740,5,1,22700,low_risk
4,10800.0,7.698,mortgage,53000,0.433962,5,1,23000,low_risk


In [4]:
# Create our features
x_cols = [i for i in df.columns if i not in ('homeowner','loan_status')]
X = df[x_cols]
    

# Create our target
y = df["loan_status"]

In [5]:
X.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000


In [6]:
 # Check the balance of our target values
y.value_counts()

low_risk     75036
high_risk     2500
Name: loan_status, dtype: int64

In [7]:
 # Create X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, 
                                                   y, 
                                                   random_state=1, 
                                                   stratify=y)
X_train.shape

(58152, 7)

In [8]:
 # Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [9]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)

In [10]:
 # Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:

#SIMPLE LOGISITIC REGRESSION
#
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_train_scaled, y_train)

LogisticRegression(random_state=1)

In [12]:
 # Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_pred)

0.9892813049736127

In [13]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[  615,    10],
       [  102, 18657]], dtype=int64)

In [14]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



In [ ]:
#OVERSAMPLING
#

In [15]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_random, y_random = ros.fit_resample(X_train_scaled, y_train)

# View the count of target classes with Counter
Counter(y_random)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [18]:
# Train the Logistic Regression model using the resampled data
model_random = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_random, y_random)

LogisticRegression(random_state=1)

In [16]:
 # Calculated the balanced accuracy score
y_random_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_random_pred)

0.9892813049736127

In [17]:
# Display the confusion matrix
confusion_matrix(y_test, y_random_pred)

array([[  615,    10],
       [  102, 18657]], dtype=int64)

In [18]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_random_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384



In [ ]:
#
#SMOTE Oversampling
#

In [19]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_smote, y_smote = SMOTE(random_state=1, sampling_strategy=1.0).fit_resample(
    X_train_scaled, y_train
)
# View the count of target classes with Counter
Counter(y_smote)

Counter({'low_risk': 56277, 'high_risk': 56277})

In [20]:
# Train the Logistic Regression model using the resampled data
model_smote = LogisticRegression(solver='lbfgs', random_state=1)
model_smote.fit(X_smote, y_smote)

LogisticRegression(random_state=1)

In [21]:
# Calculated the balanced accuracy score
y_smote_pred = model_smote.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_smote_pred)

0.9946414201183431

In [22]:
# Display the confusion matrix
confusion_matrix(y_test, y_smote_pred)

array([[  622,     3],
       [  111, 18648]], dtype=int64)

In [23]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smote_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.85      1.00      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      1.00      1.00      0.99      0.99     18759

avg / total       0.99      0.99      1.00      0.99      0.99      0.99     19384



In [ ]:
#
#UNDERSAMPLING
#

In [24]:
 # Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_cluster, y_cluster = cc.fit_resample(X_train_scaled, y_train)


# View the count of target classes with Counter
Counter(y_cluster)

Counter({'high_risk': 1875, 'low_risk': 1875})

In [25]:
# Train the Logistic Regression model using the resampled data
model_cluster = LogisticRegression(solver='lbfgs', random_state=1)
model_cluster.fit(X_cluster, y_cluster)

LogisticRegression(random_state=1)

In [26]:
# Calculate the balanced accuracy score
y_cluster_pred = model_cluster.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_cluster_pred)

0.9932279972279972

In [27]:
# Display the confusion matrix
confusion_matrix(y_test, y_cluster_pred)

array([[  620,     5],
       [  104, 18655]], dtype=int64)

In [28]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_cluster_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.99      0.99      0.92      0.99      0.99       625
   low_risk       1.00      0.99      0.99      1.00      0.99      0.99     18759

avg / total       1.00      0.99      0.99      0.99      0.99      0.99     19384



In [ ]:
#
# COMBINATION- Over and Under Sampling
#

In [29]:
 # Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN
sm = SMOTEENN(random_state=1)
X_smoteenn, y_smoteenn = sm.fit_resample(X, y)

# View the count of target classes with Counter
Counter(y_smoteenn)

Counter({'high_risk': 72651, 'low_risk': 74598})

In [30]:
# Train the Logistic Regression model using the resampled data
model_smoteenn = LogisticRegression(solver='lbfgs', random_state=1)
model_smoteenn.fit(X_smoteenn, y_smoteenn)

LogisticRegression(random_state=1)

In [31]:
 # Calculate the balanced accuracy score
y_smoteenn_pred = model.predict(X_test_scaled)
balanced_accuracy_score(y_test, y_smoteenn_pred)

0.9892813049736127

In [32]:
# Display the confusion matrix
confusion_matrix(y_test, y_smoteenn_pred)

array([[  615,    10],
       [  102, 18657]], dtype=int64)

In [33]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_smoteenn_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.86      0.98      0.99      0.92      0.99      0.98       625
   low_risk       1.00      0.99      0.98      1.00      0.99      0.98     18759

avg / total       0.99      0.99      0.98      0.99      0.99      0.98     19384

